In [ ]:
from pathlib import Path
from scipy.io import loadmat
import sys
import os


dataset_path = Path('data') / 'data.mat'
if not dataset_path.exists():
    alt = Path.cwd().parent / 'data' / 'data.mat'
    if alt.exists():
        dataset_path = alt
    else:
        raise FileNotFoundError(f"data.mat not found under {Path.cwd()} or its parent")

notebook_path = os.getcwd() 
print (f"Current notebook path: {notebook_path}")
project_root = os.path.dirname(notebook_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print (f"Added {project_root} to sys.path")

mat_data = loadmat(dataset_path)
print(mat_data.keys())

In [ ]:
import torch
import matplotlib.pyplot as plt
from utils import trainer
from utils import utils
from Systems import DynamicSystem
import Filters
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
import random

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
# Pro plnou CUDA reprodukovatelnost (volitelné, ale doporučené)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
# --------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

In [ ]:
mat_data = loadmat(dataset_path)

souradniceX_mapa = mat_data['souradniceX']
souradniceY_mapa = mat_data['souradniceY']
souradniceZ_mapa = mat_data['souradniceZ']
souradniceGNSS = mat_data['souradniceGNSS'] 

# --- KROK 1: Extrakce 1D os z 2D mřížek ---
# Získáme unikátní souřadnice pro osy X a Y.
# Pro osu X vezmeme první řádek z X matice.
# Pro osu Y vezmeme první sloupec z Y matice.
x_axis_unique = souradniceX_mapa[0, :]
y_axis_unique = souradniceY_mapa[:, 0]

print(f"Rozměry 1D osy X: {x_axis_unique.shape}")
print(f"Rozměry 1D osy Y: {y_axis_unique.shape}")
print(f"Rozměry 2D dat výšek Z: {souradniceZ_mapa.shape}")


# --- KROK 2: Vytvoření interpolačního objektu ---
# POZOR: Scipy očekává, že osy budou v pořadí (y, x), protože
# NumPy pole jsou indexována jako (řádek, sloupec), což odpovídá (y, x).
print("\nVytvářím interpolační funkci...")
terMap_interpolator = RegularGridInterpolator(
    (y_axis_unique, x_axis_unique),  # N-tice 1D os (nejprve Y, pak X)
    souradniceZ_mapa,
    bounds_error=False,  # NEVYHAZUJ CHYBU
    fill_value=np.nan
)
print("...interpolační funkce vytvořena.")

# --- KROK 3: Vytvoření finální, uživatelsky přívětivé funkce ---
def terMap(px, py):
    """
    Vypočítá nadmořskou výšku pro dané souřadnice (px, py)
    pomocí interpolace z mapy terénu.
    
    Funkce zvládne jak jednotlivé body, tak celé pole bodů.
    """
    # Spojíme vstupní body do formátu, kterému interpolátor rozumí:
    # pole o dvou sloupcích [y, x].
    points_to_query = np.column_stack((py, px))
    
    # Zavoláme interpolátor a vrátíme výsledek
    return terMap_interpolator(points_to_query)

# 4D model


In [ ]:
import torch
from math import pi
from Systems import DynamicSystemTAN

state_dim = 4
obs_dim = 3
dT = 1
q = 1

F = torch.tensor([[1.0, 0.0, dT, 0.0],
                   [0.0, 1.0, 0.0, dT],
                   [0.0, 0.0, 1.0, 0.0],
                   [0.0, 0.0, 0.0, 1.0]])

Q = q* torch.tensor([[dT**3/3, 0.0, dT**2/2, 0.0],
                   [0.0, dT**3/3, 0.0, dT**2/2],
                   [dT**2/2, 0.0, dT, 0.0],
                   [0.0, dT**2/2, 0.0, dT]])
R = torch.tensor([[3.0**2, 0.0, 0.0],
                   [0.0, 1.0**2, 0.0],
                   [0.0, 0.0, 1.0**2]])

initial_velocity_np = souradniceGNSS[:2, 1] - souradniceGNSS[:2, 0]
initial_velocity = torch.from_numpy(initial_velocity_np)

initial_position = torch.from_numpy(souradniceGNSS[:2, 0])
x_0 = torch.cat([
    initial_position,
    initial_velocity
]).float()
print(x_0)

P_0 = torch.tensor([[25.0, 0.0, 0.0, 0.0],
                    [0.0, 25.0, 0.0, 0.0],
                    [0.0, 0.0, 0.5, 0.0],
                    [0.0, 0.0, 0.0, 0.5]])

def h_nl_robust(x: torch.Tensor) -> torch.Tensor:
    # ... (implementace s clampingem, jak jsme si ukázali dříve) ...
    # Získání hranic mapy
    min_x, max_x = x_axis_unique.min(), x_axis_unique.max()
    min_y, max_y = y_axis_unique.min(), y_axis_unique.max()

    # Oříznutí pozic POUZE pro dotaz do mapy
    px_safe = x[:, 0].clone().clamp(min_x, max_x)
    py_safe = x[:, 1].clone().clamp(min_y, max_y)
    vyska_terenu_np = terMap(px_safe.detach().cpu().numpy(), py_safe.detach().cpu().numpy())
    vyska_terenu = torch.from_numpy(vyska_terenu_np).float().to(x.device)
    
    # Zbytek výpočtu s původními rychlostmi
    eps = 1e-12
    vx_w, vy_w = x[:, 2], x[:, 3]
    norm_v_w = torch.sqrt(vx_w**2 + vy_w**2).clamp(min=eps)
    cos_psi = vx_w / norm_v_w
    sin_psi = vy_w / norm_v_w
    vx_b = cos_psi * vx_w - sin_psi * vy_w
    vy_b = sin_psi * vx_w + cos_psi * vy_w
    
    result = torch.stack([vyska_terenu, vx_b, vy_b], dim=1)
    
    # Pojistka pro případ, že by terMap přesto vrátila NaN
    if torch.isnan(result).any():
        print("Varování: NaN hodnoty v měření detekovány, nahrazuji nulami.")
        result[torch.isnan(result)] = 0

    return result


x_axis_unique = souradniceX_mapa[0, :]
y_axis_unique = souradniceY_mapa[:, 0]

print("Vytvářím instanci DynamicSystemTAN...")
system_model = DynamicSystemTAN(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Q=Q.float(),
    R=R.float(),
    Ex0=x_0.float(),
    P0=P_0.float(),
    F=F.float(),
    h=h_nl_robust,
    x_axis_unique=x_axis_unique,
    y_axis_unique=y_axis_unique,
    device=device
)

In [ ]:
# TRAIN_SEQ_LEN = 300      # Krátké sekvence pro stabilní trénink (TBPTT)
# VALID_SEQ_LEN = 400      # Stejná délka pro konzistentní validaci
# # TEST_SEQ_LEN = 200      # Dlouhé sekvence pro testování generalizace

# NUM_TRAIN_TRAJ = 100   # Hodně trénovacích příkladů
# NUM_VALID_TRAJ = 10    # Dostatek pro spolehlivou validaci
# # NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

# BATCH_SIZE = 16         # Dobrý kompromis

# x_train, y_train = utils.generate_data_for_map(system_model, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
# x_val, y_val = utils.generate_data_for_map(system_model, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
# # x_test, y_test = utils.generate_data_for_map(system_model, num_trajectories=1, seq_len=TEST_SEQ_LEN)

# train_dataset = TensorDataset(x_train, y_train)
# val_dataset = TensorDataset(x_val, y_val)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from Systems import DynamicSystemTAN # Nebo jak se jmenuje tvůj model
from copy import deepcopy # Pro vytvoření kopie modelu
import numpy as np
import random # Pro náhodný výběr indexu

# --- Konfigurace (Strategie V1 - Dlouhé sekvence) ---
# Trénujeme na dlouhých sekvencích, BPTT poběží přes celou délku.
TRAIN_SEQ_LEN = 500 
# Validujeme na ještě delších, abychom ověřili časovou generalizaci.
VALID_SEQ_LEN = 600
# Kvůli rejection samplingu  bude generování dlouhých sekvencí
# trvat déle. Snížíme tedy jejich celkový počet.
NUM_TRAIN_SETS = 20 
TRAJ_PER_SET_TRAIN = 2 # Celkem 20*2 = 40 dlouhých trénovacích trajektorií
NUM_VALID_SETS = 5 
TRAJ_PER_SET_VALID = 2 # Celkem 5*2 = 10 dlouhých validačních trajektorií

# POZOR: BPTT přes 500 kroků je náročné na paměť.
# Možná budeš muset BATCH_SIZE snížit až na 1 nebo 2.
BATCH_SIZE = 4 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Načti souradniceGNSS (předpokládá se, že už existuje) ---
# souradniceGNSS = mat_data['souradniceGNSS'] # Pole tvaru [rozměr, čas]
num_gnss_points = souradniceGNSS.shape[1]
print(f"Načteno {num_gnss_points} GNSS bodů pro výběr počátečních podmínek.")

# --- Zde definuj tvůj původní system_model ---
# Předpokládám, že existuje proměnná 'system_model' s nějakými výchozími Ex0 a P0
original_system_model = system_model # Uchovej si originál

# --- Definuj varianty pro P0 (volitelné, můžeš použít jen jednu) ---
default_P0 = torch.diag(torch.tensor([25.0, 25.0, 0.5, 0.5], device=device)).float()

# --- Generování trénovacích dat ---
print(f"Generuji trénovací data (V1 strategie: {NUM_TRAIN_SETS*TRAJ_PER_SET_TRAIN} trajektorií, délka {TRAIN_SEQ_LEN})...")
all_x_train = []
all_y_train = []
for i in range(NUM_TRAIN_SETS):
    print(f"  Generuji sadu {i+1}/{NUM_TRAIN_SETS}...")

    # 1. Náhodně vyber startovní index t (musí být t+1 platný index)
    start_index = random.randint(0, num_gnss_points - 2)

    # 2. Vypočítej Ex0 z GNSS dat
    initial_pos_np = souradniceGNSS[:2, start_index]
    next_pos_np = souradniceGNSS[:2, start_index + 1]
    initial_vel_np = next_pos_np - initial_pos_np
    Ex0_sampled = torch.cat([
        torch.from_numpy(initial_pos_np),
        torch.from_numpy(initial_vel_np)
    ]).float().to(device)

    # 3. Vyber P0
    P0_current = default_P0

    # 4. Vytvoř dočasný model a generuj data
    temp_model = deepcopy(original_system_model)
    temp_model.Ex0 = Ex0_sampled
    temp_model.P0 = P0_current

    print(f"    Startovní index: {start_index}, Ex0: {Ex0_sampled.cpu().numpy()}") 

    x_batch, y_batch = utils.generate_data_for_map(
        temp_model,
        num_trajectories=TRAJ_PER_SET_TRAIN,
        seq_len=TRAIN_SEQ_LEN
    )
    all_x_train.append(x_batch)
    all_y_train.append(y_batch)

x_train = torch.cat(all_x_train, dim=0)
y_train = torch.cat(all_y_train, dim=0)
print(f"Finální trénovací data: x={x_train.shape}, y={y_train.shape}")

# --- Generování validačních dat (analogicky) ---
print(f"Generuji validační data (V1 strategie: {NUM_VALID_SETS*TRAJ_PER_SET_VALID} trajektorií, délka {VALID_SEQ_LEN})...")
all_x_val = []
all_y_val = []
for i in range(NUM_VALID_SETS):
    print(f"  Generuji sadu {i+1}/{NUM_VALID_SETS}...")
    start_index = random.randint(0, num_gnss_points - 2)
    initial_pos_np = souradniceGNSS[:2, start_index]
    next_pos_np = souradniceGNSS[:2, start_index + 1]
    initial_vel_np = next_pos_np - initial_pos_np 
    Ex0_sampled = torch.cat([
        torch.from_numpy(initial_pos_np),
        torch.from_numpy(initial_vel_np)
    ]).float().to(device)
    P0_current = default_P0 

    temp_model = deepcopy(original_system_model)
    temp_model.Ex0 = Ex0_sampled
    temp_model.P0 = P0_current

    print(f"    Startovní index: {start_index}, Ex0: {Ex0_sampled.cpu().numpy()}")

    x_batch, y_batch = utils.generate_data_for_map(
        temp_model,
        num_trajectories=TRAJ_PER_SET_VALID,
        seq_len=VALID_SEQ_LEN
    )
    all_x_val.append(x_batch)
    all_y_val.append(y_batch)

x_val = torch.cat(all_x_val, dim=0)
y_val = torch.cat(all_y_val, dim=0)
print(f"Finální validační data: x={x_val.shape}, y={y_val.shape}")

# --- Vytvoření DataLoaderů ---
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("\nDataLoadery jsou připraveny pro trénink.")

In [ ]:

# # ==============================================================================
# # 1. KONFIGURACE TESTU
# # ==============================================================================
# TEST_SEQ_LEN = 500 # Změňte zpět na 100 nebo kolik potřebujete
# NUM_TEST_TRAJ = 3
# J_SAMPLES_TEST = 25

# # ==============================================================================
# # 2. PŘÍPRAVA DAT (OPRAVENO)
# # ==============================================================================
# print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")

# # Nyní předáme oříznutou sekvenci 'u', takže i 'x' a 'y' budou mít správnou délku.
# x_test, y_test = utils.generate_data_for_map(
#     system_model, 
#     num_trajectories=NUM_TEST_TRAJ,
#     seq_len=TEST_SEQ_LEN
# )

# test_dataset = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
# print("Generování dat dokončeno.")

# print("y shape:", y_test.shape) # Mělo by být [1, 20, 1]
# print("x shape:", x_test.shape) # Mělo by být [1, 20, 3]


In [ ]:

# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# import numpy as np
# import os
# import random
# import csv
# from datetime import datetime
# import pandas as pd
# from copy import deepcopy
# from state_NN_models import StateKalmanNet

# # Nastavení seedu pro reprodukovatelnost tohoto běhu
# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)
# state_knet = StateKalmanNet(system_model, device=device, hidden_size_multiplier=12).to(device)
# print(state_knet)
# trainer.train_state_KalmanNet(
#     model=state_knet, 
#     train_loader=train_loader, 
#     val_loader=val_loader, 
#     device=device, 
#     epochs=200, 
#     lr=1e-5,
#     early_stopping_patience=40,
#     clip_grad=1.0
# )

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
from state_NN_models import StateKalmanNet_v2_4D_tan

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knet = StateKalmanNet_v2_4D_tan(system_model, device=device, hidden_size_multiplier=10, output_layer_multiplier=4, num_gru_layers=1).to(device)
print(state_knet)
trainer.train_state_KalmanNet(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=200, 
    lr=1e-4,
    early_stopping_patience=30,
    clip_grad=10.0
)

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# from state_NN_models import StateBayesianKalmanNet
# import numpy as np
# import os
# import random
# import csv
# from datetime import datetime
# import pandas as pd
# from copy import deepcopy

# model_config_phase1 = {
#     "hidden_size_multiplier": 12,
#     "output_layer_multiplier": 4,
#     "num_gru_layers": 1,
#     "init_min_dropout": 0.5,
#     "init_max_dropout": 0.8
# }

# train_config_phase1 = {
#     "total_train_iter": 1000,
#     "learning_rate": 1e-5,
#     "clip_grad": 1.0,
#     "J_samples": 10,
#     "validation_period": 20,
#     "logging_period": 2000,
#     "warmup_iterations":100 # Trénuj prvních 400 iterací jen na MSE
# }


# # Vytvoření modelu
# state_bkn_knet = StateBayesianKalmanNet(
#     system_model,
#     device=device,
#     **model_config_phase1
# ).to(device)

# # Spuštění tréninku
# # Používáme `run_training_session`, která vrací slovník s výsledky
# results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(model=state_bkn_knet,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     device=device,
#     **train_config_phase1
# )

# trained_model = results['final_model']
# print("\n" + "="*80)
# print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
# print("="*80)
# print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
# # --- Změněné klíče, aby odpovídaly return statementu ---
# print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
# print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
# print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
# print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
# print("="*80)

# print(trained_model)
# # Nyní můžeš s `trained_model` pokračovat, například ho vyhodnotit na testovací sadě.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader



# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 800 # Změňte zpět na 100 nebo kolik potřebujete
NUM_TEST_TRAJ = 5
J_SAMPLES_TEST = 25

# ==============================================================================
# 2. PŘÍPRAVA DAT (OPRAVENO)
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")

# Nyní předáme oříznutou sekvenci 'u', takže i 'x' a 'y' budou mít správnou délku.
x_test, y_test = utils.generate_data_for_map(
    system_model, 
    num_trajectories=NUM_TEST_TRAJ,
    seq_len=TEST_SEQ_LEN
)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

print("y shape:", y_test.shape) # Mělo by být [1, 20, 1]
print("x shape:", x_test.shape) # Mělo by být [1, 20, 3]



# ==============================================================================
# 3. INICIALIZACE FILTRŮ
# ==============================================================================
ukf_ideal = Filters.UnscentedKalmanFilter(system_model)
pf_sir_ideal = Filters.ParticleFilter(system_model, num_particles=10000)

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA (OPRAVENO)
# ==============================================================================
all_x_true_cpu = []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_pf_sir_ideal_cpu, all_P_hat_pf_sir_ideal_cpu = [], []
all_x_hat_classic_knet_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_knet_F3_cpu = []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

state_knet.eval()
# state_knet_F3.eval()


with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)


        # # --- A. Bayesian KalmanNet (Trajectory-wise) ---
        # ensemble_trajectories = []
        # for j in range(J_SAMPLES_TEST):
        #     state_bkn_knet.reset(batch_size=1, initial_state=initial_state)
        #     current_x_hats = []
        #     for t in range(1, TEST_SEQ_LEN):
        #         x_filtered_t, _ = state_bkn_knet.step(y_test_seq_gpu[t, :].unsqueeze(0))
        #         current_x_hats.append(x_filtered_t)
        #     ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))
        # ensemble = torch.stack(ensemble_trajectories, dim=0)
        # predictions_bkn = ensemble.mean(dim=0)
        # diff = ensemble - predictions_bkn.unsqueeze(0)
        # covariances_bkn = (diff.unsqueeze(-1) @ diff.unsqueeze(-2)).mean(dim=0)
        # full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
        # full_P_hat_bkn = torch.cat([system_model.P0.unsqueeze(0), covariances_bkn], dim=0)


        # --- B. Klasický StateKalmanNet (pouze MSE) ---
        state_knet.reset(batch_size=1, initial_state=initial_state)
        classic_knet_preds = []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t = state_knet.step(y_test_seq_gpu[t, :].unsqueeze(0))
            classic_knet_preds.append(x_filtered_t)
        full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
        
        # # --- B. Klasický StateKalmanNet (pouze MSE) ---
        # state_knet_F3.reset(batch_size=1, initial_state=initial_state)
        # classic_knet_preds_F3 = []
        # for t in range(1, TEST_SEQ_LEN):
        #     x_filtered_t = state_knet_F3.step(y_test_seq_gpu[t, :].unsqueeze(0))
        #     classic_knet_preds_F3.append(x_filtered_t)
        # full_x_hat_classic_knet_F3 = torch.cat([initial_state, torch.cat(classic_knet_preds_F3, dim=0)], dim=0)


        ukf_i_res = ukf_ideal.process_sequence(
            y_seq=y_test_seq_gpu,
            Ex0=system_model.Ex0, 
            P0=system_model.P0
        )
        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        pf_sir_i_res = pf_sir_ideal.process_sequence(y_test_seq_gpu, Ex0=system_model.Ex0,P0=system_model.P0)
        full_x_hat_pf_sir_i = pf_sir_i_res['x_filtered']
        full_P_hat_pf_sir_i = pf_sir_i_res['P_filtered']
        full_particles_history_pf_sir_i = pf_sir_i_res['particles_history']
        print(f"PF-SIR (ideální model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        # all_x_hat_knet_F3_cpu.append(full_x_hat_classic_knet_F3.cpu())
        # all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
        all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        all_x_hat_pf_sir_ideal_cpu.append(full_x_hat_pf_sir_i.cpu()); all_P_hat_pf_sir_ideal_cpu.append(full_P_hat_pf_sir_i.cpu())
        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
mse_bkn, anees_bkn = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []; mse_classic_knet = []

mse_pf_sir_ideal, anees_pf_sir_ideal = [], []
# mse_knet_F3 = []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat_full, P_hat_full):
            if x_hat_full.shape[0] != x_true.shape[0] or P_hat_full.shape[0] != x_true.shape[0]:
                 raise ValueError(f"Nesoulad délek! x_true: {x_true.shape[0]}, x_hat: {x_hat_full.shape[0]}, P_hat: {P_hat_full.shape[0]}")

            # Porovnáváme od kroku t=1
            mse = F.mse_loss(x_hat_full[1:], x_true[1:]).item()
            # ANEES se také typicky počítá od t=1 (ignoruje počáteční nejistotu P0)
            anees = utils.calculate_anees_vectorized(
                x_true[1:].unsqueeze(0),
                x_hat_full[1:].unsqueeze(0),
                P_hat_full[1:].unsqueeze(0)
            )
            return mse, anees

        # Výpočty pro všechny modely
        # mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
        # mse = F.mse_loss(all_x_hat_knet_F3_cpu[i][1:], x_true[1:]).item(); mse_knet_F3.append(mse)
        mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_sir_ideal_cpu[i], all_P_hat_pf_sir_ideal_cpu[i]); mse_pf_sir_ideal.append(mse); anees_pf_sir_ideal.append(anees)
        print("\n" + "="*80)
        print(f"trajektorie: {i + 1}/{NUM_TEST_TRAJ}")
        print("="*80)
        print("-" * 80)
        # print(f"{'Bayesian KNet (BKN)':<35} | {(mse_bkn[i]):<20.4f} | {(anees_bkn[i]):<20.4f}")
        # print(f"{'KNet F3 (pouze MSE)':<35} | {(mse_knet_F3[i]):<20.4f} | {'N/A':<20}")
        print(f"{'KNet (pouze MSE)':<35} | {(mse_classic_knet[i]):<20.4f} | {'N/A':<20}")
        print(f"{'UKF (Ideální model)':<35} | {(mse_ukf_ideal[i]):<20.4f} | {(anees_ukf_ideal[i]):<20.4f}")
        print(f"{'PF-SIR (Ideální model)':<35} | {(mse_pf_sir_ideal[i]):<20.4f} | {(anees_pf_sir_ideal[i]):<20.4f}")
        print("="*80)
      
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
# print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
# print(f"{'KNet F3 (pouze MSE)':<35} | {avg(mse_knet_F3):<20.4f} | {'N/A':<20}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print(f"{'PF-SIR (Ideální model)':<35} | {avg(mse_pf_sir_ideal):<20.4f} | {avg(anees_pf_sir_ideal):<20.4f}")
print("="*80)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch # Přidáno pro výpočet chyby a práci s tenzory

# <<< ZDE PŘEDPOKLÁDÁME EXISTENCI NÁSLEDUJÍCÍCH PROMĚNNÝCH >>>
# NUM_TEST_TRAJ = ... # Počet testovacích trajektorií
# all_x_true_cpu = [...] # Seznam pravdivých stavů
# all_x_hat_classic_knet_cpu = [...] # Seznam odhadů KNet
# all_x_hat_pf_sir_ideal_cpu = [...] # Seznam odhadů PF-SIR
# kalman_gains_history = state_knet.get_diagnostics() # Seznam tenzorů K_t z modelu KNet
# <<< KONEC PŘEDPOKLADŮ >>>
map_bounds = {
    'x_min': 1476611.42,
    'x_max': 1489541.47,
    'y_min': 6384032.63,
    'y_max': 6400441.34
}
kalman_gains_history = state_knet.get_diagnostics() # Získání historie Kalmanových zisků
index = NUM_TEST_TRAJ - 2  # Poslední trajektorie
# ==============================================================================
# 1. PŘÍPRAVA DAT (stejná jako předtím)
# ==============================================================================
x_true_plot = all_x_true_cpu[index].numpy()
# Potřebujeme tenzory pro snadný výpočet chyby
x_true_tensor = all_x_true_cpu[index]
x_knet_tensor = all_x_hat_classic_knet_cpu[index]
# x_bknet_tensor = all_x_hat_bkn_cpu[index] # Odkomentuj pokud máš BKN výsledky
x_pf_tensor = all_x_hat_pf_sir_ideal_cpu[index]

# Vypočítáme Kvadratickou Chybu (Squared Error - SE) pro každý krok a stav
squared_error = (x_knet_tensor - x_true_tensor)**2
# RMSE
rmse_per_step = torch.sqrt(squared_error).numpy()

num_steps = x_true_plot.shape[0]
time_axis = np.arange(num_steps) # Časová osa od 0 do T-1

# --- Příprava dat pro Kalmanův zisk ---
# Historie zisku je pro kroky t=1 až T-1. Potřebujeme odpovídající časovou osu.
# Délka historie zisku by měla být num_steps - 1
gain_time_axis = np.arange(1, num_steps)

# Extrakce prvků prvního sloupce Kalmanova zisku
# Předpoklad: batch_size=1 v historii, K má tvar [1, 4, 3]
try:
    gains_col0_cpu = [K[0, :, 0].numpy() for K in kalman_gains_history] # Vezmi 1. sloupec pro batch 0
    gains_col0_np = np.array(gains_col0_cpu) # Převede seznam polí na 2D pole [čas, prvek]
    plot_gains = True
except NameError:
    print("Proměnná 'kalman_gains_history' nebyla nalezena. Grafy Kalmanova zisku nebudou vykresleny.")
    plot_gains = False
except Exception as e:
    print(f"Nastala chyba při zpracování Kalmanova zisku: {e}. Grafy nebudou vykresleny.")
    plot_gains = False


state_labels = [
    'Pozice X [m]',
    'Pozice Y [m]',
    'Rychlost vX [m/s]',
    'Rychlost vY [m/s]'
]
error_labels = [
    'RMSE Pozice X [m]',
    'RMSE Pozice Y [m]',
    'RMSE Rychlost vX [m/s]',
    'RMSE Rychlost vY [m/s]'
]
# Popisky pro prvky Kalmanova zisku (první sloupec)
gain_labels = [
    'K[0,0] (Výška -> Pozice X)',
    'K[1,0] (Výška -> Pozice Y)',
    'K[2,0] (Výška -> Rychlost vX)',
    'K[3,0] (Výška -> Rychlost vY)'
]


# ==============================================================================
# 2. VYTVOŘENÍ TŘÍ SAD GRAFŮ (Trajektorie + Chyby + Zisk)
# ==============================================================================
fig1, axes1 = plt.subplots(4, 1, figsize=(12, 14), sharex=True)
fig1.suptitle('Detailní porovnání odhadů stavu v čase', fontsize=16)

fig2, axes2 = plt.subplots(4, 1, figsize=(12, 14), sharex=True)
fig2.suptitle('RMSE odhadu pro jednotlivé složky stavu v čase (KNet)', fontsize=16)

if plot_gains:
    fig3, axes3 = plt.subplots(4, 1, figsize=(12, 14), sharex=True)
    fig3.suptitle('Vývoj prvků 1. sloupce Kalmanova zisku KNet v čase', fontsize=16)

# Smyčka přes všechny 4 složky stavu
for i in range(4):
    # --- Graf 1: Trajektorie ---
    ax1 = axes1[i]
    ax1.plot(time_axis, x_true_plot[:, i], 'r-', linewidth=2.0, label='Referenční hodnota')
    ax1.plot(time_axis, x_knet_tensor[:, i].numpy(), 'g--', linewidth=1.5, label='Odhad KNet')
    # ax1.plot(time_axis, x_bknet_tensor[:, i].numpy(), 'b-.', linewidth=1.5, label='Odhad BKNet')
    ax1.plot(time_axis, x_pf_tensor[:, i].numpy(), 'm:', linewidth=1.5, label='Odhad PF-SIR')
    ax1.set_ylabel(state_labels[i])
    ax1.grid(True)
    ax1.legend()


    if i == 0: # Graf pro Pozici X
        ax1.axhline(map_bounds['x_min'], color='grey', linestyle=':', linewidth=1.5, label='Hranice mapy X')
        ax1.axhline(map_bounds['x_max'], color='grey', linestyle=':', linewidth=1.5) # Není třeba label znovu
        print(f"INFO: Přidávám hranice X ({map_bounds['x_min']:.2f}, {map_bounds['x_max']:.2f}) do grafu Pozice X.")
    elif i == 1: # Graf pro Pozici Y
        ax1.axhline(map_bounds['y_min'], color='grey', linestyle=':', linewidth=1.5, label='Hranice mapy Y')
        ax1.axhline(map_bounds['y_max'], color='grey', linestyle=':', linewidth=1.5)
        print(f"INFO: Přidávám hranice Y ({map_bounds['y_min']:.2f}, {map_bounds['y_max']:.2f}) do grafu Pozice Y.")
    

    # --- Graf 2: Chyba (RMSE) ---
    ax2 = axes2[i]
    ax2.plot(time_axis, rmse_per_step[:, i], 'b-', linewidth=1.5, label=f'RMSE KNet (Avg: {np.mean(rmse_per_step[1:, i]):.2f})') # Průměr bez počátečního bodu
    ax2.set_ylabel(error_labels[i])
    ax2.grid(True)
    ax2.legend()
    # ax2.set_ylim(bottom=0)

    # --- Graf 3: Kalmanův zisk (pokud jsou data k dispozici) ---
    if plot_gains and gains_col0_np.shape[0] == len(gain_time_axis): # Kontrola délky
         ax3 = axes3[i]
         ax3.plot(gain_time_axis, gains_col0_np[:, i], 'k-', linewidth=1.5, label=f'{gain_labels[i]} (Avg: {np.mean(gains_col0_np[:, i]):.4f})')
         ax3.set_ylabel(gain_labels[i])
         ax3.grid(True)
         ax3.legend()
    elif plot_gains:
         print(f"Varování: Délka historie zisku ({gains_col0_np.shape[0]}) neodpovídá časové ose ({len(gain_time_axis)}). Graf zisku pro prvek {i} nebude vykreslen.")


# Nastavení popisků pro sdílené osy X
axes1[-1].set_xlabel('Časový krok [s]')
axes2[-1].set_xlabel('Časový krok [s]')
if plot_gains:
    axes3[-1].set_xlabel('Časový krok [s]')
# Zlepšíme rozložení
fig1.tight_layout(rect=[0, 0.03, 1, 0.96])
fig2.tight_layout(rect=[0, 0.03, 1, 0.96])
if plot_gains:
    fig3.tight_layout(rect=[0, 0.03, 1, 0.96])

plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# %matplotlib widget

# # --- Předpokládáme, že tyto proměnné již existují z vašeho vyhodnocení ---
# # all_x_true_cpu: Seznam s pravdivou trajektorií
# # full_x_hat_classic_knet: Tenzor s odhady z klasického KNetu
# # full_x_hat_bkn: Tenzor s odhady z Bayesian KNetu (předpoklad)
# # souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa: Data mapy
# # terMap: Vaše interpolační funkce

# # --- Krok 1: Příprava dat ---
# x_true_plot = all_x_true_cpu[0].numpy()
# x_knet_plot = full_x_hat_classic_knet.cpu().numpy()

# print(f"Tvar skutečné trajektorie: {x_true_plot.shape}")
# print(f"Tvar odhadnuté trajektorie (KNet): {x_knet_plot.shape}")

# # --- Krok 2: Vytvoření 3D grafu ---
# fig = plt.figure(figsize=(14, 12))
# ax = fig.add_subplot(111, projection='3d')

# # Vykreslení povrchu terénu (volitelné)
# ax.plot_surface(souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa, 
#                   rstride=100, cstride=100, cmap='terrain', alpha=0.3)

# # --- Krok 3: Vykreslení trajektorií ---

# # A. Skutečná (referenční) trajektorie
# px_true = x_true_plot[:, 0]
# py_true = x_true_plot[:, 1]
# pz_true = terMap(px_true, py_true)
# ax.plot(px_true, py_true, pz_true, 'r-', linewidth=3, label='Referenční trajektorie')

# # B. Odhadnutá trajektorie z KalmanNetu
# px_knet = x_knet_plot[:, 0]
# py_knet = x_knet_plot[:, 1]
# pz_knet = terMap(px_knet, py_knet)
# ax.plot(px_knet, py_knet, pz_knet, 'g--', linewidth=3, label='Odhad KNet')


# # --- Krok 4: Finalizace grafu ---
# ax.plot([px_true[0]], [py_true[0]], [pz_true[0]], 
#         'o', color='black', markersize=10, label='Start')

# ax.set_xlabel('Souřadnice X [m]')
# ax.set_ylabel('Souřadnice Y [m]')
# ax.set_zlabel('Nadmořská výška Z [m]')

# # Upravíme název, aby zahrnoval všechny modely
# ax.set_title('Porovnání referenční trajektorie a odhadů KNet/BKN') 
# ax.legend()
# ax.grid(True)

# ax.view_init(elev=30., azim=-60)

# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# %matplotlib widget

# # --- Předpokládáme, že tyto proměnné již existují z vašeho vyhodnocení ---
# # all_x_true_cpu: Seznam s pravdivou trajektorií
# # full_x_hat_classic_knet: Tenzor s odhady z klasického KNetu
# # full_x_hat_bkn: Tenzor s odhady z Bayesian KNetu (předpoklad)
# # souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa: Data mapy
# # terMap: Vaše interpolační funkce

# # --- Krok 1: Příprava dat ---
# x_true_plot = all_x_true_cpu[0].numpy()

# print(f"Tvar skutečné trajektorie: {x_true_plot.shape}")


# # --- Krok 2: Vytvoření 3D grafu ---
# fig = plt.figure(figsize=(14, 12))
# ax = fig.add_subplot(111, projection='3d')

# # Vykreslení povrchu terénu (volitelné)
# ax.plot_surface(souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa, 
#                   rstride=100, cstride=100, cmap='terrain', alpha=0.3)

# # --- Krok 3: Vykreslení trajektorií ---

# # A. Skutečná (referenční) trajektorie
# px_true = x_true_plot[:, 0]
# py_true = x_true_plot[:, 1]
# pz_true = terMap(px_true, py_true)
# ax.plot(px_true, py_true, pz_true, 'r-', linewidth=3, label='Referenční trajektorie')


# # --- Krok 4: Finalizace grafu ---
# ax.plot([px_true[0]], [py_true[0]], [pz_true[0]], 
#         'o', color='black', markersize=10, label='Start')

# ax.set_xlabel('Souřadnice X [m]')
# ax.set_ylabel('Souřadnice Y [m]')
# ax.set_zlabel('Nadmořská výška Z [m]')

# # Upravíme název, aby zahrnoval všechny modely
# ax.set_title('Porovnání referenční trajektorie a odhadů KNet/BKN') 
# ax.legend()
# ax.grid(True)

# ax.view_init(elev=30., azim=-60)

# plt.show()